<a href="https://colab.research.google.com/github/jihun5/data-preprocessing_/blob/main/%EB%AA%A8%EB%8D%B8%EC%9E%91%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

bank_df = pd.read_csv('bank-prep.csv', sep=',')
bank_df.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,dec,feb,jan,jul,jun,mar,may,nov,oct,sep
0,58,0,2143,1,0,5,261,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
1,36,0,265,1,1,5,348,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
2,25,0,-7,1,0,5,365,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
3,53,0,-3,0,0,5,1666,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
4,24,0,-103,1,1,5,145,1,-1,0,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
# yes와 no의 개수 차이로 인하여 정확한 측정에 어려움이 있을 수 있다.
# 다운샘플링 진행
!pip show imbalanced-learn

Name: imbalanced-learn
Version: 0.10.1
Summary: Toolbox for imbalanced dataset in machine learning.
Home-page: https://github.com/scikit-learn-contrib/imbalanced-learn
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: joblib, numpy, scikit-learn, scipy, threadpoolctl
Required-by: 


In [ ]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.8.6
aiosignal                        1.3.1
alabaster                        0.7.13
albumentations                   1.3.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.1.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.13.1
backcall                         0.2.0
beautifulsoup4                   4.11.2
bidict                           0.22.1
b

In [ ]:
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
X = np.array(bank_df.drop('y', axis=1))
Y = np.array(bank_df[['y',]])
print(np.sum(Y == 1), np.sum(Y == 0))

sampler = RandomUnderSampler(random_state=42)
X, Y = sampler.fit_resample(X,Y)
print(np.sum( Y==1), np.sum(Y==0))

820 6113
820 820


In [ ]:
# 오버 샘플링
from imblearn.over_sampling import RandomOverSampler
'''
X = np.array(bank_df.drop('y', axis=1))
Y = np.array(bank_df[['y']])
print(np.sum(Y==1), np.sum(Y==0))

sampler = RandomOverSampler(random_state=42)
X, Y = sampler.fit_resample(X,Y)
print(np.sum(Y==1), np.sum(Y==0))
'''

"\nX = np.array(bank_df.drop('y', axis=1))\nY = np.array(bank_df[['y']])\nprint(np.sum(Y==1), np.sum(Y==0))\n\nsampler = RandomOverSampler(random_state=42)\nX, Y = sampler.fit_resample(X,Y)\nprint(np.sum(Y==1), np.sum(Y==0))\n"

복사시키지 않고 보완해서 데이터를 증가시키는 방법 SMOTE

SMOTE는 K-NN법 아이디어를 사용한다. 데이터 점 사이의 이웃 근접도는 유클리드 거리를 사용하여 두 점 간의 직선거리를 계산한다.

SMOTE는 수가 적은 클래스에 주목해서 어떤 데이터 점과 근접한 데이터 점 간에 새로운 데이터 점 간에 새로운 데이터 점을 생성하는 것이다.

In [ ]:
from sklearn.model_selection import KFold
from sklearn import tree
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=10, shuffle=True)
scores = []  # 점수를 저장할 리스트

for train_id, test_id in kf.split(X):
    x_train, x_test = X[train_id], X[test_id]
    y_train, y_test = Y[train_id], Y[test_id]

    clf = tree.DecisionTreeClassifier()
    clf.fit(x_train, y_train)
    pred_y = clf.predict(x_test)

    # accuracy_score 함수에 실제 레이블과 예측 레이블을 모두 전달
    acc_score = accuracy_score(y_test, pred_y)
    scores.append(acc_score)  # 점수를 리스트에 추가

scores = np.array(scores)
print(scores.mean(), scores.std())

0.7804878048780488 0.027675135897341985


In [ ]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

print(recall_score(Y[test_id], pred_y))
print(precision_score(Y[test_id], pred_y))

0.8352941176470589
0.8160919540229885


In [ ]:
print(clf)

DecisionTreeClassifier()


In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'criterion' : ['entropy', 'gini'],
    'max_depth' : [2,4,6,8,10],
    'min_samples_leaf' : [10,20,30,40,50],
}

clf_gs = GridSearchCV(tree.DecisionTreeClassifier(), params, cv=KFold(n_splits=10, shuffle=True), scoring='accuracy')
clf_gs.fit(X, Y)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
             estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_depth': [2, 4, 6, 8, 10],
                         'min_samples_leaf': [10, 20, 30, 40, 50]},
             scoring='accuracy')

In [ ]:
print(clf_gs.best_score_)
print(clf_gs.best_params_)

0.8048780487804879
{'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 20}


In [ ]:
clf_best = tree.DecisionTreeClassifier(criterion = 'gini', max_depth=8, min_samples_leaf=10)
clf_best.fit(X,Y)
print(clf_best.feature_importances_)

[2.99835241e-02 0.00000000e+00 4.55619334e-02 6.10541159e-02
 0.00000000e+00 2.62043691e-02 4.76733314e-01 2.75710515e-02
 1.83269206e-01 0.00000000e+00 0.00000000e+00 1.97524994e-02
 1.67314193e-03 2.02005131e-04 0.00000000e+00 1.33136920e-03
 0.00000000e+00 0.00000000e+00 5.27129616e-02 7.72359134e-03
 0.00000000e+00 0.00000000e+00 1.53479523e-03 0.00000000e+00
 7.47406997e-04 0.00000000e+00 4.90999733e-02 1.28817581e-02
 1.96298400e-03 0.00000000e+00 0.00000000e+00]


In [ ]:
bank_df.columns

Index(['age', 'default', 'balance', 'housing', 'loan', 'day', 'duration',
       'campaign', 'pdays', 'previous', 'y', 'divorced', 'married', 'single',
       'primary', 'secondary', 'tertiary', 'cellular', 'telephone', 'unknown',
       'apr', 'aug', 'dec', 'feb', 'jan', 'jul', 'jun', 'mar', 'may', 'nov',
       'oct', 'sep'],
      dtype='object')

In [14]:
bank_df = pd.read_csv('bank.csv', sep=',')
bank_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,NaN,5,may,261,1,-1,0,NaN,no
1,36,technician,single,secondary,no,265,yes,yes,NaN,5,may,348,1,-1,0,NaN,no
2,25,blue-collar,married,secondary,no,-7,yes,no,NaN,5,may,365,1,-1,0,NaN,no
3,53,technician,married,secondary,no,-3,no,no,NaN,5,may,1666,1,-1,0,NaN,no
4,24,technician,single,secondary,no,-103,yes,yes,NaN,5,may,145,1,-1,0,NaN,no


In [16]:
# JOB 통합 및 새로운 특장량 생성
# 문자열 값의 통합
bank_df.loc[
    (bank_df['job'] == 'management') |
    (bank_df['job'] == 'technician') |
    (bank_df['job'] == 'blue-collar') |
    (bank_df['job'] == 'admin.') |
    (bank_df['job'] == 'services') |
    (bank_df['job'] == 'self-employed') |
    (bank_df['job'] == 'entrepreneur') |
    (bank_df['job'] == 'housemaid'), 'job2'] = 'worker'
bank_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,job2
0,58,management,married,tertiary,no,2143,yes,no,NaN,5,may,261,1,-1,0,NaN,no,worker
1,36,technician,single,secondary,no,265,yes,yes,NaN,5,may,348,1,-1,0,NaN,no,worker
2,25,blue-collar,married,secondary,no,-7,yes,no,NaN,5,may,365,1,-1,0,NaN,no,worker
3,53,technician,married,secondary,no,-3,no,no,NaN,5,may,1666,1,-1,0,NaN,no,worker
4,24,technician,single,secondary,no,-103,yes,yes,NaN,5,may,145,1,-1,0,NaN,no,worker


In [17]:
bank_df.loc[(bank_df['job'] == 'retired') |
            (bank_df['job'] == 'unemployed') |
            (bank_df['job'] == 'student'), 'job2'] = 'non-worker'

bank_df.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,job2
0,58,management,married,tertiary,no,2143,yes,no,NaN,5,may,261,1,-1,0,NaN,no,worker
1,36,technician,single,secondary,no,265,yes,yes,NaN,5,may,348,1,-1,0,NaN,no,worker
2,25,blue-collar,married,secondary,no,-7,yes,no,NaN,5,may,365,1,-1,0,NaN,no,worker
3,53,technician,married,secondary,no,-3,no,no,NaN,5,may,1666,1,-1,0,NaN,no,worker
4,24,technician,single,secondary,no,-103,yes,yes,NaN,5,may,145,1,-1,0,NaN,no,worker
5,60,retired,married,tertiary,no,100,no,no,NaN,5,may,528,1,-1,0,NaN,no,non-worker
6,55,technician,married,secondary,no,1205,yes,no,NaN,5,may,158,2,-1,0,NaN,no,worker
7,54,management,married,secondary,no,282,yes,yes,NaN,5,may,154,1,-1,0,NaN,no,worker
8,55,services,divorced,secondary,no,91,no,no,NaN,5,may,349,1,-1,0,NaN,no,worker
9,56,admin.,married,secondary,no,45,no,no,NaN,5,may,1467,1,-1,0,NaN,yes,worker


In [18]:
bank_df.loc[(bank_df['month'] == 'jan') |
            (bank_df['month'] == 'feb') |
            (bank_df['month'] == 'mar'), 'month2'] = '1Q'

In [19]:
bank_df.loc[(bank_df['month'] == 'apr') |
            (bank_df['month'] == 'may') |
            (bank_df['month'] == 'jun'), 'month'] = '2Q'
bank_df.loc[(bank_df['month'] == 'jul') |
            (bank_df['month'] == 'aug') |
            (bank_df['month'] == 'sep'), 'month2'] = '3Q'

In [20]:
# 수치통합
bank_df.loc[bank_df['day'] <= 10, 'day2'] = 'early'
bank_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,job2,month2,day2
0,58,management,married,tertiary,no,2143,yes,no,NaN,5,2Q,261,1,-1,0,NaN,no,worker,NaN,early
1,36,technician,single,secondary,no,265,yes,yes,NaN,5,2Q,348,1,-1,0,NaN,no,worker,NaN,early
2,25,blue-collar,married,secondary,no,-7,yes,no,NaN,5,2Q,365,1,-1,0,NaN,no,worker,NaN,early
3,53,technician,married,secondary,no,-3,no,no,NaN,5,2Q,1666,1,-1,0,NaN,no,worker,NaN,early
4,24,technician,single,secondary,no,-103,yes,yes,NaN,5,2Q,145,1,-1,0,NaN,no,worker,NaN,early


In [21]:
bank_df.loc[(bank_df['month'] == 'oct') |
            (bank_df['month'] == 'nov') |
            (bank_df['month'] == 'dec'), 'month2'] = '4Q'

In [22]:
bank_df.loc[bank_df['duration'] < 300, 'duration2'] = 'short'
bank_df.loc[bank_df['duration'] >= 300, 'duration2'] = 'long'
bank_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,...,duration,campaign,pdays,previous,poutcome,y,job2,month2,day2,duration2
0,58,management,married,tertiary,no,2143,yes,no,NaN,5,...,261,1,-1,0,NaN,no,worker,NaN,early,short
1,36,technician,single,secondary,no,265,yes,yes,NaN,5,...,348,1,-1,0,NaN,no,worker,NaN,early,long
2,25,blue-collar,married,secondary,no,-7,yes,no,NaN,5,...,365,1,-1,0,NaN,no,worker,NaN,early,long
3,53,technician,married,secondary,no,-3,no,no,NaN,5,...,1666,1,-1,0,NaN,no,worker,NaN,early,long
4,24,technician,single,secondary,no,-103,yes,yes,NaN,5,...,145,1,-1,0,NaN,no,worker,NaN,early,short


In [23]:
# previous 통합 및 새로운 특징량 생성
bank_df.loc[bank_df['previous'] < 1, 'previous2'] = 'zero'
bank_df.loc[bank_df['previous'] >= 1, 'previous2'] = 'one-more'
bank_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,...,campaign,pdays,previous,poutcome,y,job2,month2,day2,duration2,previous2
0,58,management,married,tertiary,no,2143,yes,no,NaN,5,...,1,-1,0,NaN,no,worker,NaN,early,short,zero
1,36,technician,single,secondary,no,265,yes,yes,NaN,5,...,1,-1,0,NaN,no,worker,NaN,early,long,zero
2,25,blue-collar,married,secondary,no,-7,yes,no,NaN,5,...,1,-1,0,NaN,no,worker,NaN,early,long,zero
3,53,technician,married,secondary,no,-3,no,no,NaN,5,...,1,-1,0,NaN,no,worker,NaN,early,long,zero
4,24,technician,single,secondary,no,-103,yes,yes,NaN,5,...,1,-1,0,NaN,no,worker,NaN,early,short,zero


In [24]:
bank_df.loc[bank_df['pdays'] < 0, 'pdays2'] = 'less'
bank_df.loc[bank_df['pdays'] >= 0, 'pdays2'] = 'more'
bank_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,...,pdays,previous,poutcome,y,job2,month2,day2,duration2,previous2,pdays2
0,58,management,married,tertiary,no,2143,yes,no,NaN,5,...,-1,0,NaN,no,worker,NaN,early,short,zero,less
1,36,technician,single,secondary,no,265,yes,yes,NaN,5,...,-1,0,NaN,no,worker,NaN,early,long,zero,less
2,25,blue-collar,married,secondary,no,-7,yes,no,NaN,5,...,-1,0,NaN,no,worker,NaN,early,long,zero,less
3,53,technician,married,secondary,no,-3,no,no,NaN,5,...,-1,0,NaN,no,worker,NaN,early,long,zero,less
4,24,technician,single,secondary,no,-103,yes,yes,NaN,5,...,-1,0,NaN,no,worker,NaN,early,short,zero,less


In [25]:
# 분석 데이터 세트의 작성 및 추가
bank_df_job2 = pd.get_dummies(bank_df['job2'])
bank_df_month2 = pd.get_dummies(bank_df['month2'])
bank_df_day2 = pd.get_dummies(bank_df['day2'])
bank_df_duration2 = pd.get_dummies(bank_df['duration2'])
bank_df_previous2 = pd.get_dummies(bank_df['previous2'])
bank_df_pdays2 = pd.get_dummies(bank_df['pdays2'])

In [26]:
tmp5 = pd.concat([bank_df, bank_df_job2], axis=1)
tmp6 = pd.concat([tmp5, bank_df_month2], axis=1)
tmp7 = pd.concat([tmp6, bank_df_day2], axis=1)
tmp8 = pd.concat([tmp7, bank_df_duration2], axis=1)
tmp9 = pd.concat([tmp8, bank_df_previous2], axis=1)
bank_df_new2 = pd.concat([tmp9, bank_df_pdays2], axis=1)
bank_df_new2.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,...,1Q,3Q,4Q,early,long,short,one-more,zero,less,more
0,58,management,married,tertiary,no,2143,yes,no,NaN,5,...,0,0,0,1,0,1,0,1,1,0
1,36,technician,single,secondary,no,265,yes,yes,NaN,5,...,0,0,0,1,1,0,0,1,1,0
2,25,blue-collar,married,secondary,no,-7,yes,no,NaN,5,...,0,0,0,1,1,0,0,1,1,0
3,53,technician,married,secondary,no,-3,no,no,NaN,5,...,0,0,0,1,1,0,0,1,1,0
4,24,technician,single,secondary,no,-103,yes,yes,NaN,5,...,0,0,0,1,0,1,0,1,1,0


In [27]:
bank_df_new2.to_csv('bank-prep2.csv', index=False)

In [29]:
# feature_selection을 통한 변수 사용 유무 확인
from sklearn.feature_selection import SelectKBest
selector = SelectKBest(k=5) # 결과(분류)에 영향을 미치는 변수 5개를 선택
selector.fit(X,Y)
mask = selector.get_support()

print(bank_df.drop('y', axis=1).columns)
print(mask)

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'job2', 'month2', 'day2', 'duration2',
       'previous2', 'pdays2'],
      dtype='object')
[False False False  True False False  True False False  True False False
 False False False False  True False  True False False False False False
 False False False False False False False]
